In [1]:
# !pip install -U flash-attn --no-build-isolation
!pip install transformers accelerate optimum

In [2]:
# print(torch.version.cuda)

In [3]:
from transformers import AutoProcessor, BarkModel
import optimum
import os
import shlex
from optimum.bettertransformer import BetterTransformer
import torch
import scipy
from scipy.io.wavfile import write as write_wav
from IPython.display import Audio

/home/changgeng/anaconda3/envs/torchi/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
commandline_args = os.environ.get('COMMANDLINE_ARGS', "--skip-torch-cuda-test --no-half")
shlex.split(commandline_args)

['--skip-torch-cuda-test', '--no-half']

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_id = "suno/bark-small"
model = BarkModel.from_pretrained(model_id, torch_dtype=torch.float32).to(device)
# model = BarkModel.from_pretrained("suno/bark-small", torch_dtype=torch.float16, attn_implementation="flash_attention_2").to(device)
model =  BetterTransformer.transform(model)


/home/changgeng/anaconda3/envs/torchi/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


In [6]:
processor = AutoProcessor.from_pretrained("suno/bark")
model = BarkModel.from_pretrained("suno/bark")
voice_preset = "v2/de_speaker_2"

In [7]:
inputs = processor(
    """
    Wer reitet so spät durch Nacht und Wind?
Es ist der Vater mit seinem Kind;
Er hat den Knaben wohl in dem Arm,
Er fasst ihn sicher, er hält ihn warm. [clears throat]

Mein Sohn, was birgst du so bang dein Gesicht? –
Siehst, Vater, du den Erlkönig nicht?
Den Erlenkönig mit Kron' und Schweif? –
Mein Sohn, es ist ein Nebelstreif. [clears throat]

'Du liebes Kind, komm, geh mit mir!
Gar schöne Spiele spiel’ ich mit dir;
Manch' bunte Blumen sind an dem Strand,
Meine Mutter hat manch gülden Gewand.' – [clears throat]

Mein Vater, mein Vater, und hörest du nicht,
Was Erlenkönig mir leise verspricht? –
Sei ruhig, bleibe ruhig, mein Kind;
In dürren Blättern säuselt der Wind. – 
""", voice_preset=voice_preset)
# inputs


In [8]:
audio_array = model.generate(**inputs, do_sample=True).to(device)
audio_array = audio_array.cpu().numpy().squeeze()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


In [11]:
# audio_array = audio_array.squeeze()
sample_rate = model.generation_config.sample_rate

In [12]:
version_2_path = "Version_02_gedicht.wav"
write_wav(version_2_path, sample_rate, audio_array)
# scipy.io.wavfile.write(version_2_path, rate=24000, data=audio_array)
display(Audio(version_2_path, autoplay=True))

In [ ]:
# Bark hat auch eine generate Funktion
# https://stackoverflow.com/questions/76266716/text-to-speech-convert-to-wav-in-python